In [61]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from math import cos, sin, pi
import time
from IPython import display
import numpy as np
import matplotlib.cm as cm

In [13]:
class Obstacle:
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        
    def lines_collide(self, x0_start, y0_start, x0_end, y0_end, x1_start, y1_start, x1_end, y1_end):
        x0_delta = x0_end - x0_start
        y0_delta = y0_end - y0_start
        x1_delta = x1_end - x1_start
        y1_delta = y1_end - y1_start
        
        d = x1_delta * y0_delta - x0_delta * y1_delta
        if d == 0:
            return False
        
        s = (1 / d) * ((x0_start - x1_start) * y0_delta - (y0_start - y1_start) * x0_delta)
        t = (1 / d) * -(-(x0_start - x1_start) * y1_delta + (y0_start - y1_start) * x1_delta)
        return s >= 0 and s <= 1 and t >= 0 and t <= 1
    
    def collides_with(self, x_start, y_start, x_end, y_end):
        return self.lines_collide(self.x, self.y, self.x + self.width, self.y, x_start, y_start, x_end, y_end) or \
            self.lines_collide(self.x, self.y, self.x, self.y + self.height, x_start, y_start, x_end, y_end) or \
            self.lines_collide(self.x + self.width, self.y, self.x + self.width, self.y + self.height, x_start, y_start, x_end, y_end) or \
            self.lines_collide(self.x, self.y + self.height, self.x + self.width, self.y + self.height, x_start, y_start, x_end, y_end)          
    
    def draw(self, ax):
        ax.add_patch(
            patches.Rectangle(
                (self.x, self.y),
                self.width, 
                self.height,
            )
        )

In [14]:
class Joint:
    def __init__(self, length, min_angle, max_angle):
        self.length = length
        self.min_angle = min_angle
        self.max_angle = max_angle
        self.angle = self.min_angle
        self.childs = []
        self.line = None
        
    def add_child(self, new_joint):
        self.childs.append(new_joint)
        
    def collides_with(self, obstacles, offset_x, offset_y, offset_angle):
        x_end = offset_x + cos((self.angle + offset_angle) / 180 * pi) * self.length
        y_end = offset_y + sin((self.angle + offset_angle) / 180 * pi) * self.length   
                
        for obstacle in obstacles:
            if obstacle.collides_with(offset_x, offset_y, x_end, y_end):
                return True
        
        for child in self.childs:
            if child.collides_with(obstacles, x_end, y_end, (self.angle + offset_angle) + 180):
                return True
        return False
    
    def draw(self, ax, offset_x, offset_y, offset_angle):
        x_end = offset_x + cos((self.angle + offset_angle) / 180 * pi) * self.length
        y_end = offset_y + sin((self.angle + offset_angle) / 180 * pi) * self.length       
        if self.line is None:
            self.line = ax.plot([offset_x, x_end], [offset_y, y_end], color='k', linestyle='-', linewidth=2)[0]
        else:
            self.line.set_xdata([offset_x, x_end])
            self.line.set_ydata([offset_y, y_end])
        
        for child in self.childs:
            child.draw(ax, x_end, y_end, (self.angle + offset_angle) + 180)
            
    def reset(self):
        self.angle = self.min_angle
        for child in self.childs:
            child.reset()
            
    def advance(self, step_size):
        for child in self.childs:
            if child.advance(step_size):
                return True
            
        self.angle += step_size
        if self.angle >= self.max_angle:
            self.angle = self.min_angle
            return False
        else:
            return True
    
    def get_angles(self):
        angles = [self.angle] 
        for child in self.childs:
            angles += child.get_angles()
        return angles

In [75]:
obstacles = [Obstacle(10, 50, 20, 20), Obstacle(80, 20, 10, 50)]

In [82]:
root = Joint(40, 0, 180)
root.add_child(Joint(30, 0, 360))
#root.collides_with(obstacles, 0, 0)

In [83]:
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')
fig.canvas.draw()
background = fig.canvas.copy_from_bbox(ax.bbox)
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)

               
def draw():    
    global ax
    #ax.clear()
    #ax = fig.add_subplot(111, aspect='equal')
    fig.canvas.restore_region(background)
    for obstacle in obstacles: 
        obstacle.draw(ax)
    root.draw(ax, 50, 0, 0) 
    fig.canvas.blit(ax.bbox) 
    fig.canvas.draw()


FigureCanvasNbAgg()

In [84]:
for i in range(300):
    root.advance(10)  
draw()
go_on = True
root.reset()
configuration_space_in = [[] for i in range(len(root.get_angles()))]
configuration_space_out = []

while go_on:
    go_on = root.advance(3)    
    for i, angle in enumerate(root.get_angles()):
        configuration_space_in[i].append(angle)
    configuration_space_out.append(0 if root.collides_with(obstacles, 50, 0, 0) else 1)

x=np.unique(configuration_space_in[1])
y=np.unique(configuration_space_in[0])
X,Y = np.meshgrid(x,y)

Z=np.array(configuration_space_out).reshape(len(y),len(x))
#print(Z)
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')
ax.pcolormesh(X,Y,Z, cmap=cm.gray)

FigureCanvasNbAgg()

In [7]:
root.reset()
for i in range(1000):
    root.advance(20)
    draw()
    display.clear_output()
    print(root.collides_with(obstacles, 50, 0, 0))
    time.sleep(0.4)

False


KeyboardInterrupt: 